In [ ]:
import matplotlib,aplpy
from astropy.io import fits
import common_functions as cf
import matplotlib.pyplot as plt
from astropy.wcs import WCS
from astropy.table import Table

In [ ]:
font = {'size'   : 14, 'family' : 'serif', 'serif' : 'cm'}
plt.rc('font', **font)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['axes.linewidth'] = 1

In [ ]:
def channel_maps(cubefile,imagefile,src_cat,ra_cen,dec_cen,fov,save_figs=True):

    #Load the cube
    cube,cube_ra,cube_dec,cube_vel = read_fitscube(cubefile)

    #Make map for each channel
    for i in range(len(cube_vel)):
        #Initialise background figure
        f = aplpy.FITSFigure(imagefile,figsize=(10,10),dimensions=[0,1])

        f.recenter(ra_cen,dec_cen, height=fov, width=fov)

        #Aplpy cannot properly read the header and slice the cube
        #So we have to read it with astropy and pass it to aplpy
        hdu = fits.open(cubefile)
        wcs = WCS(hdu[0].header,naxis=2)
        data = hdu[0].data[i]
        hdu = fits.PrimaryHDU(data)
        hdu.header.update(wcs.to_header())
        
        src_tab = Table.read(src_cat)
        
        rms = numpy.mean(src_tab['rms'])

        #Overlay HI channel contours
        f.show_contour(hdu,
                       colors=['black'],levels=numpy.array([-2])*rms,linestyle='--')
        f.show_contour(hdu,
                       colors=['black','blue','purple','red','darkorange'],levels=numpy.array([2,4,8,16,32])*rms)

        #Set colour stretch of background image
        f.show_colorscale(cmap='Greys',vmin=0.0001,vmax=5.,stretch='log')

        #Add grid lines
        f.add_grid()
        f.grid.set_color('black')

        #Add velcoity label
        f.add_label(0.8,0.95,str(int(numpy.round(cube_vel[i],0)))+' km/s',horizontalalignment='left',
                    relative='axes', color='r', size='large')

        #Save
        if save_figs:
            f.savefig(f'channel_maps/HCG{HCG}/chnmap_'+str(i)+'.png')

        #Close figure
        f.close()

In [ ]:
HCGs = Table.read('../output/HCGs.vo', format='votable')
HCGs.add_index('HCG')

In [ ]:
HCG = 31
channel_maps('SoFiA_masks/HCG{0}/HCG{0}_HI.fits'.format(HCG),
             'DECaLS_images/r_fits/HCG{0}_r.fits'.format(HCG), 
             'SoFiA_masks/HCG{0}/HCG{0}_cat.xml'.format(HCG), 
             HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], 0.1)